In [ ]:
import yfinance as yf
import pandas as pd
import sys
from pathlib import Path
from utils import preencher_nulos

root_path = str(Path.cwd().parent)
if root_path not in sys.path:
    sys.path.append(root_path)

In [5]:
# 1. Parâmetros
ativo = "ITUB3.SA"
data_inicial = "2023-01-01"
data_final = "2026-02-15"

# 2. Download dos dados
tabela_cotacao = yf.download(
    ativo,
    start=data_inicial,
    end=data_final,
    progress=False,
    multi_level_index=False,
)

# 3. Cálculos de variação
tabela_cotacao["Variacao"] = tabela_cotacao["Close"].diff()
tabela_cotacao["Var%"] = tabela_cotacao["Close"].pct_change() * 100

# 4. Transformando o índice em datetime e garantindo que ele seja o Index
tabela_cotacao.index = pd.to_datetime(tabela_cotacao.index)
tabela_cotacao.columns.name = None

col_nulo = ["Variacao", "Var%"]

tabela_cotacao = preencher_nulos(tabela_cotacao, col_nulo, 0)

tabela_cotacao.head(10)

,Close,High,Low,Open,Volume,Variacao,Var%
Date,,,,,,,
2023-01-02,14.359387,14.486581,14.178641,14.479886,544066,0.000000,0.000000
2023-01-03,14.084925,14.392865,13.971120,14.359392,899262,-0.274463,-1.911382
2023-01-04,14.098313,14.312533,14.031370,14.125091,908666,0.013389,0.095056
2023-01-05,14.513362,14.587000,14.111702,14.171951,1218768,0.415049,2.943959
2023-01-06,14.700799,14.767742,14.479884,14.546829,1122123,0.187437,1.291479
2023-01-09,14.740963,14.841378,14.566910,14.653937,1122916,0.040164,0.273210
2023-01-10,14.975269,15.022130,14.526747,14.687411,698267,0.234306,1.589491
2023-01-11,15.350147,15.397009,14.881543,14.961876,942316,0.374878,2.503313
2023-01-12,15.323376,15.450570,14.968576,15.283210,1505983,-0.026772,-0.174406


In [6]:
caminho_raiz = Path.cwd().parent

pasta_parquet = caminho_raiz / "arquivos_parquet"
pasta_parquet.mkdir(parents=True, exist_ok=True)

caminho_arquivo = pasta_parquet / "dados_base.parquet"
tabela_cotacao.to_parquet(caminho_arquivo)

print(f"Arquivo salvo em: {caminho_arquivo}")

Arquivo salvo em: c:\Users\Pc Gamer\OneDrive\FIAP\Machine Learnig Engineering\Fase - 04\fiap_tech_challenge_fase4\arquivos_parquet\dados_base.parquet
